In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModel
from transformers import AutoModelForSequenceClassification, AutoConfig, Trainer, TrainingArguments
import torch

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_parquet(r"input_data\ccd_labeled_train_validate.parquet")

print(df.shape)
display(df.head())                    

In [ ]:
# testing - check for duplicates or errors in labels
sorted(df['product'].unique())

In [ ]:
# testing - check for duplicates or errors in labels

plt.title("Frequency of Classes")
df['product'].value_counts().sort_index().plot(kind='bar', rot=70)
plt.show()

plt.title("Frequency of Classes")
df['product'].value_counts(ascending=True).plot.barh()
plt.show()

In [ ]:
aa = pd.DataFrame()
aa['Words per cust utterance'] = df['text'].str.split().apply(len)
aa['product'] = df['product'].copy()
aa.boxplot("Words per cust utterance", by="product", rot=70)
plt.show()

# Transformer

In [ ]:
dft = df[['text', 'product']].copy()
dft.rename(columns={'product': 'label_name'}, inplace=True)
display(dft.head(2))
print()
label_name = sorted(dft['label_name'].unique())
label = [i for i in range(len(label_name))]

label2id = {lbl_name: lbl for lbl_name, lbl in zip(label_name, label)}
id2label = {lbl: lbl_name for lbl_name, lbl in zip(label_name, label)}
for k, v in label2id.items():
    print(f"{str(k):15}", ' --- ', v)
# for k, v in id2label.items():
#     print(f"{str(k):15}", ' --- ', v)

In [ ]:
dft['label'] =dft['label_name'].apply(lambda x: label2id[x])
dft = dft[['text', 'label', 'label_name']]
dft.sample(5)

In [ ]:
# model_ckpt = "distilbert-base-uncased"
model_ckpt = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)


text = "I love machine learning! Tokenization is awesome!!"
encoded_text = tokenizer(text)
print(encoded_text)

In [ ]:
len(tokenizer.vocab), tokenizer.vocab_size, tokenizer.model_max_length

In [ ]:
dft.columns

In [ ]:
train, test = train_test_split(dft, test_size=0.3, stratify=dft['label_name'], random_state=42)
test, validation = train_test_split(test, test_size=1/3, stratify=test['label_name'], random_state=42)

train.shape, validation.shape, test.shape

In [ ]:
dataset = DatasetDict(
    {'train':Dataset.from_pandas(train, preserve_index=False),
     'test':Dataset.from_pandas(test, preserve_index=False),
     'validation': Dataset.from_pandas(validation, preserve_index=False)
     }
)
dataset

# Tokenise Data

In [ ]:
print(dataset['train'][0], '\n\n') 
print(dataset['train'][1])

In [ ]:
def tokenize(batch):
    temp = tokenizer(batch['text'], padding=True, truncation=True)
    return temp

print(tokenize(dataset['train'][:2]))

In [ ]:
ccd_encoded = dataset.map(tokenize, batched=True, batch_size=None)
ccd_encoded

# Build Model

In [ ]:
# model = AutoModel.from_pretrained(model_ckpt)
# print(model)
# print(model.config.id2label)
# print(model.config)

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoConfig

num_labels = len(label2id)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
config = AutoConfig.from_pretrained(model_ckpt, label2id=label2id, id2label=id2label)
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, config=config).to(device)
device

In [ ]:
# print(model.config.id2label)

In [ ]:
# print(model)
print(model.config)

In [ ]:
from transformers import TrainingArguments

batch_size = 64
training_dir = "trained_model/bert_base_train_dir"

training_args = TrainingArguments( output_dir=training_dir,
                                  overwrite_output_dir = True,
                                  num_train_epochs = 1,  # change num epochs
                                  learning_rate = 2e-5,
                                  per_device_train_batch_size = batch_size,
                                  per_device_eval_batch_size = batch_size,
                                  weight_decay = 0.01,
                                  evaluation_strategy = 'epoch',
                                  disable_tqdm = False
                                 )

In [ ]:
# use sklearn to build compute metrics
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)

    return {"accuracy": acc, "f1": f1}

In [ ]:
trainer = Trainer(model=model, args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset = ccd_encoded['train'],
                  eval_dataset = ccd_encoded['validation'],
                  tokenizer = tokenizer)

In [ ]:
trainer.train()

# Model Evaluation

In [ ]:
preds_output = trainer.predict(ccd['test'])
preds_output.metrics

In [ ]:
y_pred = np.argmax(preds_output.predictions, axis=1)
y_true = ccd_encoded['test'][:]['label']

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))

In [ ]:
label2id

In [ ]:
# plot confusion matrix
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(5,5))
sns.heatmap(cm, annot=True, xticklabels=label2id.keys(), yticklabels=label2id.keys(), fmt='d', cbar=False, cmap='Reds')
plt.ylabel("Actual")
plt.xlabel("Predicted")
plt.show()

# Build Prediction Function and Store Model

In [ ]:
text = "I want to open a fixed deposit account now, thank you."

def get_prediction(text):
    input_encoded = tokenizer(text, return_tensors='pt').to(device)

    with torch.no_grad():
        outputs = model(**input_encoded)

    logits = outputs.logits

    pred = torch.argmax(logits, dim=1).item()
    return id2label[pred]

get_prediction(text)

In [ ]:
trainer.save_model("model_trained/bert-base-uncased-product_clr-model")

In [ ]:
# use pipeline for prediciton
from transformers import pipeline

classifier = pipeline('text-classification', model= 'model_trained/bert-base-uncased-sentiment-model')

classifier(["I want to open a fixed deposit account now, thank you.", "your atm swallowed my card", "Why is your loan interest rate so high?"])